# Project Euler 21 - 30

## Problem 31

[Coin sums](https://projecteuler.net/problem=31)

> <div class="problem_content" role="problem">
<p>In England the currency is made up of pound, £, and pence, p, and there are eight coins in general circulation:</p>
<blockquote>1p, 2p, 5p, 10p, 20p, 50p, £1 (100p) and £2 (200p).</blockquote>
<p>It is possible to make £2 in the following way:</p>
<blockquote>1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p</blockquote>
<p>How many different ways can £2 be made using any number of coins?</p>
</div>

This is a standard DP (dynamic programming) problem using a recursive relation.

Let $d(t, c)$ be the number of different ways to pay $t$ pence with coins up to $c$-th kind. For example, $d(11, 3)$ represents number of ways to pay $11$ pence with coins $\{1\textrm{p}, 2\textrm{p}, 5\textrm{p}\}$. There is recursive relation between $c+1$ and $c$ by

$$ d(t, c+1) = \sum_{k = 0, 1, 2, \dots} d(t - k \, v_{c+1}, c) $$

where $v_c$ represents value of the $c$-th coin. We also know that $d(0, 0) = 1$ and $d(t, 0) = 0$ for $t > 0$ as boundary conditions.

Our goal is to pay $200$ pence with $8$ different coins $(1, 2, 5, 10, 20, 50, 100, 200)$, which is $d(200, 8)$. We assign two dimensional state space starting from 

In [15]:
import collections

def prob031_dp(target=200):
    coins = [1, 2, 5, 10, 20, 50, 100, 200]
    n = len(coins)
    d = collections.defaultdict(int)
    d[(0, 0)] = 1
    for c in range(n):
        v = coins[c]
        for t in range(target + 1):
            d[(t, c+1)] = sum(d[(t - i * v, c)] for i in range(t // v + 1))
    return d[(200, n)]

In [16]:
%time prob031_dp()

CPU times: user 24.4 ms, sys: 1.12 ms, total: 25.6 ms
Wall time: 25.6 ms


73682

Memoisation version.

In [5]:
import functools

@functools.lru_cache(maxsize=1000000)
def _helper(total, coins):
    if total < 0:
        return 0
    elif total == 0:
        return 1
    elif len(coins) == 0:
        return 0
    else:
        return _helper(total, coins[1:]) + _helper(total - coins[0], coins)

def prob031(total=200):
    return _helper(total, (200, 100, 50, 20, 10, 5, 2, 1))

In [6]:
%time prob031()

CPU times: user 12.7 ms, sys: 2.39 ms, total: 15.1 ms
Wall time: 16 ms


73682

## Problem 32

[Pandigital products](https://projecteuler.net/problem=32)

> <div class="problem_content" role="problem">
<p>We shall say that an <var>n</var>-digit number is pandigital if it makes use of all the digits 1 to <var>n</var> exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.</p>
<p>The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.</p>
<p>Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.</p>
<div class="note">HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.</div>
</div>

Let $d(x)$ be the numbers of digits of an integer $x$.

$$    10^{d(x)-1} \leq x < 10^{d(x)}  $$

Then the product of two integers, $x$ and $y4, is

$$ 10^{d(x) + d(y) -2} \leq x y < 10^{d(x) + d(y)}, $$

or

$$  d(x) + d(y) - 1 \leq  d(x y)  \leq d(x) + d(y).  $$

We also impose that digits of $x$, $y$ and $x y$ forms pandigital (1 through 9) in total.

$$    d(x) + d(y) + d(x y) = 9. $$

From these relations, and that $d(z)$ is an integer, we have

$$    d(x) + d(y) = 5  $$

For $x \leq y$, we have $(d(x), d(y)) = (1, 4)$ or $(2, 3)$.

In [24]:
import collections

def integerdigits(n):
    return [int(c) for c in str(n)]

def is_pandigital(x, y):
    xs = sorted(i for n in (x, y, x * y) for i in integerdigits(n)) 
    return xs == list(range(1, 10))

def prob032():
    set1 = {(x, y, x * y) for x in range(10) for y in range(1000, 10000) if is_pandigital(x, y)}
    set2 = {(x, y, x * y) for x in range(10, 100) for y in range(100, 1000) if is_pandigital(x, y)}
    print(set1| set2)
    return sum({p for _, _, p in set1 | set2})

In [25]:
%time prob032()

{(4, 1963, 7852), (28, 157, 4396), (18, 297, 5346), (39, 186, 7254), (4, 1738, 6952), (27, 198, 5346), (12, 483, 5796), (42, 138, 5796), (48, 159, 7632)}
CPU times: user 2.43 s, sys: 4.06 ms, total: 2.44 s
Wall time: 2.44 s


45228

## Problem 33

[Digit cancelling fractions](https://projecteuler.net/problem=33)

> <div class="problem_content" role="problem">
<p>The fraction <sup>49</sup>/<sub>98</sub> is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that <sup>49</sup>/<sub>98</sub> = <sup>4</sup>/<sub>8</sub>, which is correct, is obtained by cancelling the 9s.</p>
<p>We shall consider fractions like, <sup>30</sup>/<sub>50</sub> = <sup>3</sup>/<sub>5</sub>, to be trivial examples.</p>
<p>There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator.</p>
<p>If the product of these four fractions is given in its lowest common terms, find the value of the denominator.</p>
</div>

Straightforward implementation works.

In [79]:
import fractions
import functools
import operator

def is_curious(numer, denom):
    assert numer < denom
    assert 10 <= numer < 100
    assert 10 <= denom < 100
    ns = integerdigits(numer)
    ds = integerdigits(denom)
    cancelling = set(ns) & set(ds)
    if len(cancelling) == 1 and 0 not in cancelling:
        v = cancelling.pop()
        ns.remove(v)
        ds.remove(v)
        n, d = ns[0], ds[0]
        return n * denom == d * numer
    else:
        return False

def product(iterable):
    return functools.reduce(operator.mul, iterable, 1)

def prob033():
    fracs = [fractions.Fraction(numer, denom) 
             for numer in range(10, 100)
             for denom in range(numer + 1, 100)
             if is_curious(numer, denom)]
    return product(fracs).denominator

In [80]:
%time prob033()

CPU times: user 127 ms, sys: 3.89 ms, total: 130 ms
Wall time: 132 ms


100

## Problem 34

[Digit factorials](https://projecteuler.net/problem=34)

> <div class="problem_content" role="problem">
<p>145 is a curious number, as 1! + 4! + 5! = 1 + 24 + 120 = 145.</p>
<p>Find the sum of all numbers which are equal to the sum of the factorial of their digits.</p>
<p class="info">Note: as 1! = 1 and 2! = 2 are not sums they are not included.</p>
</div>

## Problem 35

[Circular primes](https://projecteuler.net/problem=35)

> <div class="problem_content" role="problem">
<p>The number, 197, is called a circular prime because all rotations of the digits: 197, 971, and 719, are themselves prime.</p>
<p>There are thirteen such primes below 100: 2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, and 97.</p>
<p>How many circular primes are there below one million?</p>
</div>

## Problem 36

[Double-base palindromes](https://projecteuler.net/problem=36)

> <div class="problem_content" role="problem">
<p>The decimal number, 585 = 1001001001<sub>2</sub> (binary), is palindromic in both bases.</p>
<p>Find the sum of all numbers, less than one million, which are palindromic in base 10 and base 2.</p>
<p class="info">(Please note that the palindromic number, in either base, may not include leading zeros.)</p>
</div>

## Problem 37

[Truncatable primes](https://projecteuler.net/problem=37)

> <div class="problem_content" role="problem">
<p>The number 3797 has an interesting property. Being prime itself, it is possible to continuously remove digits from left to right, and remain prime at each stage: 3797, 797, 97, and 7. Similarly we can work from right to left: 3797, 379, 37, and 3.</p>
<p>Find the sum of the only eleven primes that are both truncatable from left to right and right to left.</p>
<p class="info">NOTE: 2, 3, 5, and 7 are not considered to be truncatable primes.</p>
</div>

## Problem 38

[Pandigital multiples](https://projecteuler.net/problem=38)

> <div class="problem_content" role="problem">
<p>Take the number 192 and multiply it by each of 1, 2, and 3:</p>
<blockquote>192 × 1 = 192<br/>
192 × 2 = 384<br/>
192 × 3 = 576</blockquote>
<p>By concatenating each product we get the 1 to 9 pandigital, 192384576. We will call 192384576 the concatenated product of 192 and (1,2,3)</p>
<p>The same can be achieved by starting with 9 and multiplying by 1, 2, 3, 4, and 5, giving the pandigital, 918273645, which is the concatenated product of 9 and (1,2,3,4,5).</p>
<p>What is the largest 1 to 9 pandigital 9-digit number that can be formed as the concatenated product of an integer with (1,2, ... , <var>n</var>) where <var>n</var> &gt; 1?</p>
</div>

## Problem 39

[Integer right triangles](https://projecteuler.net/problem=39)

> <div class="problem_content" role="problem">
<p>If <i>p</i> is the perimeter of a right angle triangle with integral length sides, {<i>a</i>,<i>b</i>,<i>c</i>}, there are exactly three solutions for <i>p</i> = 120.</p>
<p>{20,48,52}, {24,45,51}, {30,40,50}</p>
<p>For which value of <i>p</i> ≤ 1000, is the number of solutions maximised?</p>
</div>

## Problem 40

[Champernowne's constant](https://projecteuler.net/problem=40)

> <div class="problem_content" role="problem">
<p>An irrational decimal fraction is created by concatenating the positive integers:</p>
<p style="text-align:center;">0.12345678910<span style="color:#dd0000;font-size:14pt;">1</span>112131415161718192021...</p>
<p>It can be seen that the 12<sup>th</sup> digit of the fractional part is 1.</p>
<p>If <i>d</i><sub><i>n</i></sub> represents the <i>n</i><sup>th</sup> digit of the fractional part, find the value of the following expression.</p>
<p style="text-align:center;"><i>d</i><sub>1</sub> × <i>d</i><sub>10</sub> × <i>d</i><sub>100</sub> × <i>d</i><sub>1000</sub> × <i>d</i><sub>10000</sub> × <i>d</i><sub>100000</sub> × <i>d</i><sub>1000000</sub></p>
</div>